In [1]:
%matplotlib inline

In [2]:
import math
import webdataset as wds
import scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice
import ray
from ray.data import Dataset
import braceexpand
import requests
import random
import json

In [3]:
ray.init()

Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2023-05-11 23:14:13,835	INFO worker.py:1607 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.10.6
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8265


In [4]:
!gsutil ls gs://ocro-iaa/lin/ | shardsum

gs://ocro-iaa/lin/lin-{000000..000703}.tar


In [5]:
source = list(braceexpand.braceexpand("gs://ocro-iaa/lin/lin-{000000..000703}.tar"))
destination = "gs://ocro-tempout"

In [6]:
for s in source:
    !gsutil cat $s | tar tvf - | sed 5q
    break

-r--r--r-- bigdata/bigdata 17251 2023-05-11 06:56 67656e65736973776869746566616d30306d616c74676f6f67/00000.lin.json
-r--r--r-- bigdata/bigdata     2 2023-05-11 06:56 67656e65736973776869746566616d30306d616c74676f6f67/00001.lin.json
-r--r--r-- bigdata/bigdata   204 2023-05-11 06:56 67656e65736973776869746566616d30306d616c74676f6f67/00002.lin.json
-r--r--r-- bigdata/bigdata   153 2023-05-11 06:56 67656e65736973776869746566616d30306d616c74676f6f67/00003.lin.json
-r--r--r-- bigdata/bigdata   153 2023-05-11 06:56 67656e65736973776869746566616d30306d616c74676f6f67/00004.lin.json
tar: stdout: write error


In [7]:
!env | grep KEYS

In [8]:
dataset = ray.data.read_webdataset(source, parallelism=9999)

2023-05-11 23:14:22,325	WARNING dataset.py:255 -- Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.

Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode


In [9]:
def page_classifier(lin, do_plot=False, **kw):
    ys = []
    for text, bbox in lin:
        if bbox is None: continue
        bbox = map(int, bbox.split())
        x0, y0, x1, y1 = bbox
        ys.append(float(y1))
    if len(ys) < 10:
        return "mostly-empty"
    xs = np.linspace(0, 1, len(ys))
    ys = ndi.median_filter(ys, 5)
    ys = ndi.gaussian_filter(ys, 20.0)
    ys -= np.amin(ys)
    if np.amax(ys) < 1e-3:
        return "mostly-empty"
    ys /= np.amax(ys)
    ysm = np.maximum.accumulate(ys)
    delta = ys - ysm
    single = np.all(delta > -0.1)
    if single:
        result = "single"
    elif np.mean(ys[:len(ys)//4]) > 0.5:
        result = "backwards"
    else:
        result = "multi"
    color = dict(single="green", backwards="red", multi="blue")[result]
    if do_plot:
        plt.plot(xs, ys, color=color, **kw)
    return result

def balance_samples(sample):
    page = sample["lin.json"]
    kind = page_classifier(page)
    if kind == "single":
        if random.uniform(0, 1) < 0.1:
            return True
        else:
            return False
    elif kind == "multi":
        return True
    else:
        return False

In [10]:
filtered_dataset = dataset.filter(balance_samples)

2023-05-11 23:14:22,650	WARNING dataset.py:4352 -- The `map`, `flat_map`, and `filter` operations are unvectorized and can be very slow. If you're using a vectorized transformation, consider using `.map_batches()` instead.


In [11]:
shuffled_dataset = filtered_dataset.random_shuffle()

In [12]:
shuffled_dataset.write_webdataset("gs://ocro-tempout/")

2023-05-11 23:14:23,814	INFO streaming_executor.py:91 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[ReadWebDataset->Filter->RandomShuffle] -> TaskPoolMapOperator[Write]
2023-05-11 23:14:23,816	INFO streaming_executor.py:92 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-05-11 23:14:23,818	INFO streaming_executor.py:94 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


- ReadWebDataset->Filter->RandomShuffle 1:   0%|          | 0/704 [00:00<?, ?it/s]

ShuffleMap 2:   0%|          | 0/704 [00:00<?, ?it/s]

ShuffleReduce 3:   0%|          | 0/704 [00:00<?, ?it/s]

Running 0:   0%|          | 0/704 [00:00<?, ?it/s]

(map pid=579808) /tmp/ipykernel_576487/3262028565.py:14: RuntimeWarning: invalid value encountered in divide
(map pid=578096) /tmp/ipykernel_576487/3262028565.py:14: RuntimeWarning: invalid value encountered in divide [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(map pid=578102) /tmp/ipykernel_576487/3262028565.py:14: RuntimeWarning: invalid value encountered in divide
(map pid=578105) /tmp/ipykernel_576487/3262028565.py:14: RuntimeWarning: invalid value encountered in divide
(map pid=578105) /tmp/ipykernel_576487/3262028565.py:14: RuntimeWarning: invalid value encountered in divide
(map pid=578095) /tmp/ipykernel_576487/3262028565.py:14: RuntimeWarning: invalid value encountered in divide
(map pid=578100) /tmp/ipykernel_576487/3262028565.py:14: RuntimeWarning: invalid value encountered in divide
(map pid=5780